In [1]:
from spotipy.oauth2 import SpotifyOAuth
from sqlalchemy import create_engine
import pandas as pd
import sqlalchemy
import sqlite3
import spotipy
import math

## 1. Extract

In [2]:
scope = "user-library-read"
username = "Hunor Tot-Bagi"
client_id = "ff5d45edced9494aaee5427d666e5a13"
client_secret = "5123bcdbaeac4450a24889181a2960b2"
redirect_uri = "http://localhost:8000/callback"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, username=username, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri))

data = sp.current_user_saved_tracks(limit=50)

In [3]:
# Creating empty lists to store the data
song_names = []
artist_names = []
song_duration = []
popularity = []
release_date = []
added_at = []

# Loops through the 'items' in the 'data' and appends the selected attributes to corresponding lists
for song in data['items']:
    song_names.append(song['track']['name'])
    artist_names.append(song['track']['album']['artists'][0]['name'])
    song_duration.append(song['track']['duration_ms'])
    popularity.append(song['track']['popularity'])
    release_date.append(song['track']['album']['release_date'])
    added_at.append(song['added_at'])
    
# Creates a dictionary using the lists created earlier and assigns each list to a key in the dictionary
song_dict = {
    "song_name" : song_names,
    "artist_name" : artist_names,
    "song_duration" : song_duration,
    "popularity" : popularity,
    "release_date" : release_date,
    "added_at" : added_at
}

# Converts the dictionary to a Pandas DataFrame
song_df = pd.DataFrame(song_dict, columns = ['song_name',
                                             'artist_name',
                                             'song_duration', 
                                             'popularity',
                                             'release_date',
                                             'added_at'])

# Renames the column headers of the DataFrame to be more visually appealing
song_df = song_df.rename(columns={'song_name': 'Song name',
                                  'artist_name': 'Artist name',
                                  'song_duration': 'Miliseconds',
                                  'popularity': 'Popularity',
                                  'release_date': 'Song release date',
                                  'added_at' : 'Date added to playlist'})

song_df.head(5)

,Song name,Artist name,Miliseconds,Popularity,Song release date,Date added to playlist
0,E.T.,Katy Perry,206373,68,2010-08-24,2023-05-01T20:23:08Z
1,Talk Dirty (feat. 2 Chainz),Jason Derulo,177685,71,2013-09-10,2023-05-01T20:21:24Z
2,Trapenele,Tóth Andi,114710,52,2023-04-09,2023-04-29T13:30:04Z
3,Nem és nem,Tóth Andi,137763,45,2022-04-22,2023-04-29T13:30:02Z
4,Néztek,Tóth Andi,147164,53,2021-06-18,2023-04-29T13:30:01Z


## 2. Transform

In [4]:
# Add new columns for song duration in minutes and seconds, rounded down to the nearest integer
song_df['Minutes'] = (song_df['Miliseconds'] / 60000).apply(math.floor)
song_df['Seconds'] = ((song_df['Miliseconds'] / 1000) % 60).apply(math.floor)

In [5]:
song_df.head(5)

,Song name,Artist name,Miliseconds,Popularity,Song release date,Date added to playlist,Minutes,Seconds
0,E.T.,Katy Perry,206373,68,2010-08-24,2023-05-01T20:23:08Z,3,26
1,Talk Dirty (feat. 2 Chainz),Jason Derulo,177685,71,2013-09-10,2023-05-01T20:21:24Z,2,57
2,Trapenele,Tóth Andi,114710,52,2023-04-09,2023-04-29T13:30:04Z,1,54
3,Nem és nem,Tóth Andi,137763,45,2022-04-22,2023-04-29T13:30:02Z,2,17
4,Néztek,Tóth Andi,147164,53,2021-06-18,2023-04-29T13:30:01Z,2,27


In [6]:
# Define a function that takes a DataFrame as input
def check_if_valid_data(df):
    
    # Check if DataFrame is empty
    if df.empty:
        print("1. No songs downloaded.")
    else:
        print('1. Data Frame is not empty.')
    
    # Primary Key Check
    # The "Date added to playlist" column is a suitable primary key candidate 
    # Since it is impossible for two songs to be added to the playlist at the exact same time
    if pd.Series(df['Date added to playlist']).is_unique:
        print('2. Primary Key column is unique.')
    else:
        raise Exception('2. Primary Key Check is violated.')
        
    # Check for Null values in DataFrame
    if df.isnull().values.any():
        raise Exception('3. Null valued found')
    else:
        print('3. No Null value found.')
        
check_if_valid_data(song_df)

1. Data Frame is not empty.
2. Primary Key column is unique.
3. No Null value found.


## 3. Load

In [7]:
# Set the location of the database file
DATABASE_LOCATION = "sqlite:///my_favorite_songs.sqlite"

# Create a database engine object using SQLAlchemy
engine = sqlalchemy.create_engine(DATABASE_LOCATION)

# Connect to the database using sqlite3
conn = sqlite3.connect('my_favorite_songs.sqlite')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Define an SQL query to create a table if it doesn't exist already
sql_query = """
CREATE TABLE IF NOT EXISTS my_played_tracks(
    song_names VARCHAR(200),
    artist_names VARCHAR(200),
    song_duration VARCHAR(200),
    popularity VARCHAR(200),
    release_date VARCHAR(200),
    added_at VARCHAR(200),
)
"""

In [8]:
# Try to write a pandas dataframe called 'song_df' to a SQL table called 'my_favorite_songs'
try:
    song_df.to_sql('my_favorite_songs', engine, index=False, if_exists='append')
    
# If the table already exists, print an error message
except:
    print('Data already exists in the database')
    
conn.close()
print('Closed the database successfully')

Closed the database successfully
